In [ ]:
import os
import fsspec
import pandas as pd
os.getcwd()

In [ ]:
file_list = os.listdir('data')
print(file_list)

In [ ]:
fs_write = fsspec.filesystem(
    's3',
    profile='osn-hytest-internal',  ## aws profile name for bucket you are writing data to
    client_kwargs={'endpoint_url': 'https://usgs.osn.mghpcc.org'}
)

In [ ]:
fs_write.ls('hytest-internal/nhm_prms_v1_1_gridmet/raw_data')

In [ ]:
complete_df = pd.DataFrame(columns=['huc12', 'date'])
for f in file_list:
    var_name = f.split('_mo')[0].split('huc12_')[1]
    print(var_name)
    df = pd.read_csv(os.path.join('data', f), dtype={'huc12': 'string'})
    #print(df.head())
    long_df = pd.melt(df, id_vars='huc12', var_name='date', value_name=var_name)
    complete_df = complete_df.merge(long_df, on=['huc12', 'date'], how='right')
    #print(complete_df.head())
    fname=f'hytest-internal/nhm_prms_v1_1_gridmet/raw_data/{f}'
    print(f'writing {fname}')
    outfile = fs_write.open(fname, mode='w')
    print(df.head())
    df.to_csv(outfile, index=False)
complete_df['date'] =  pd.to_datetime(complete_df['date'], format='%Y-%m')
complete_fname='hytest-internal/nhm_prms_v1_1_gridmet/nhm_prms_v1_1_gridmet.csv'
print(f'writing {complete_fname}')
#complete_df['huc12'] = complete_df["huc12"].map(str)
complete_outfile = fs_write.open(complete_fname, mode='w')
complete_df.to_csv(complete_outfile, index=False)

In [ ]:
fs_write.ls('hytest-internal/nhm_prms_v1_1_gridmet')